In [ ]:
# 데이터 처리 및 기타 작업을 위한 라이브러리들을 불러옵니다.
import pandas as pd                                         # 데이터프레임 구조를 다루는 데 필수적인 라이브러리
import numpy as np                                          # 숫자 연산을 위한 라이브러리
import datetime as dt                                       # 날짜와 시간을 다루는 라이브러리
import json                                                 # JSON 형식의 데이터를 처리하는 라이브러리

import taipy as tp
from taipy import Config, Orchestrator, Gui

In [ ]:
tp.get_scenarios()

In [1]:
from algos.algos import *

KeyboardInterrupt: 

In [ ]:
all_scenarios = tp.get_scenarios()
# print_object_attributes(all_scenarios[0])

for scenario in all_scenarios:
    print(f"Scenario ID: {scenario.id}")
    

In [ ]:
all_scenarios[0].preprocessed_dataset.read().head()

In [ ]:
models = {
    "baseline",
    "logistic_regression",
    "random_forest",
    "xgboost",
    # "tree": train_model_decision_tree,
}

In [ ]:
# model = "xgboost"
# tp_exp = "scenario.metrics_"+model+".read()"
# metrics_xgboost = eval(tp_exp)
# metrics_xgboost

In [ ]:
ftpn_df = pd.DataFrame()


for scenario in all_scenarios:
    for model in models:
        metrics = eval("scenario.metrics_" + model + ".read()")
        dict_ftpn = metrics["dict_ftpn"]
        
        # 새로운 행을 딕셔너리로 만듭니다.
        # 기존 dict_ftpn에 추가 정보를 더합니다.
        new_row = {
            'fn': dict_ftpn.get('fn'),
            'fp': dict_ftpn.get('fp'),
            'tn': dict_ftpn.get('tn'),
            'tp': dict_ftpn.get('tp'),
            'model': model,
            'scenario_name': scenario._config_id,
            'scenario_id': scenario.id,
            'feature_selection_info_id': scenario.feature_selection_info.id,
            'threshold_baseline_id': eval("scenario.threshold_" + model + ".id"),
            'train_parameters_info_id': eval("scenario.train_parameters_info_" + model + ".id")
        }

        # ftpn_df에 새로운 행을 추가합니다.
        # pandas.concat을 사용해 행을 추가하는 것이 일반적으로 append보다 효율적입니다.
        ftpn_df = pd.concat([ftpn_df, pd.DataFrame([new_row])], ignore_index=True)

# 'fn', 'fp', 'tn', 'tp' 컬럼을 가장 앞으로 이동시키기
# 루프 이후에 한 번만 처리합니다.
target_cols = ['fn', 'fp', 'tn', 'tp', 'model']
other_cols = [col for col in ftpn_df.columns if col not in target_cols]
ftpn_df = ftpn_df[target_cols + other_cols]

# 결과 확인
print(ftpn_df.head())

In [ ]:
import os
import pandas as pd

# 저장할 파일 경로
file_path = 'data/result/summary_metrics_ftpn_df.csv'

# 저장할 DataFrame (예시 ftpn_df)
# 실제 코드에서는 이전에 생성한 ftpn_df를 사용합니다.
# ftpn_df = ...

# 폴더가 없으면 생성
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# 파일이 존재하는지 확인
file_exists = os.path.isfile(file_path)

# 파일이 이미 존재하면 헤더 없이 'a' (append) 모드로 저장
if file_exists:
    ftpn_df.to_csv(file_path, mode='a', header=False, index=False)
    print(f"Data has been appended to '{file_path}'.")
else:
    # 파일이 존재하지 않으면 헤더와 함께 'w' (write) 모드로 저장
    ftpn_df.to_csv(file_path, mode='w', header=True, index=False)
    print(f"A new file has been created and saved at '{file_path}'.")

In [ ]:
import pandas as pd

# Create an empty DataFrame to store the feature importance data
feature_importance_df = pd.DataFrame()

# Iterate through all scenarios and models
for scenario in all_scenarios:
    for model in models:
        # Construct the string for reading the feature importance
        feature_importance_str = f"scenario.feature_importance_{model}.read()"

        try:
            # Read the feature importance data for the specific model and scenario
            importance_data = eval(feature_importance_str)

            # Add columns for scenario and model identifiers
            importance_data['model'] = model
            importance_data['scenario_id'] = scenario.id
            importance_data['scenario_name'] = scenario._config_id
            
            # Add the requested info
            importance_data['feature_selection_info_id'] = scenario.feature_selection_info.id
            importance_data['threshold_id'] = eval(f"scenario.threshold_{model}.id")
            importance_data['train_parameters_info_id'] = eval(f"scenario.train_parameters_info_{model}.id")


            # Append the data to the main DataFrame
            feature_importance_df = pd.concat([feature_importance_df, importance_data], ignore_index=True)

        except AttributeError:
            print(f"No feature importance data found for model '{model}' in scenario '{scenario._config_id}'. Skipping.")
            continue

# Display the first few rows of the final DataFrame
print(feature_importance_df.head())

In [ ]:
# feature_importance_df


In [ ]:
import os
import pandas as pd

# 저장할 파일 경로
file_path = 'data/result/summary_feature_importance_df.csv'

# 저장할 DataFrame (예시 feature_importance_df)
# 실제 코드에서는 이전에 생성한 feature_importance_df를 사용합니다.
# feature_importance_df = ...

# 폴더가 없으면 생성
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# 파일이 존재하는지 확인
file_exists = os.path.isfile(file_path)

# 파일이 이미 존재하면 헤더 없이 'a' (append) 모드로 저장
if file_exists:
    feature_importance_df.to_csv(file_path, mode='a', header=False, index=False)
    print(f"Data has been appended to '{file_path}'.")
else:
    # 파일이 존재하지 않으면 헤더와 함께 'w' (write) 모드로 저장
    feature_importance_df.to_csv(file_path, mode='w', header=True, index=False)
    print(f"A new file has been created and saved at '{file_path}'.")

In [ ]:
import os
import shutil

# 원본 파일이 있는 디렉토리
source_dir = 'user_data/jsons'
# 복사할 대상 디렉토리
destination_dir = 'data/result/jsons'

# 대상 디렉토리가 없으면 생성
os.makedirs(destination_dir, exist_ok=True)

# 원본 디렉토리의 모든 파일에 대해 반복
for filename in os.listdir(source_dir):
    # JSON 파일만 선택
    if filename.endswith('.json'):
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_dir, filename)
        
        # 파일 복사. 대상 경로에 파일이 이미 존재하면 덮어쓰기
        shutil.copy2(source_path, destination_path)
        print(f"'{source_path}' -> '{destination_path}' (파일 덮어쓰기 완료)")

print("모든 JSON 파일 복사가 완료되었습니다.")

In [ ]:
# # print_object_attributes(all_scenarios[0])
# scenario = all_scenarios[0]

In [ ]:
# scenario.feature_importance_logistic_regression.read().head()

In [ ]:
# model = "logistic_regression"
# eval("scenario.threshold_"+model+".id")

In [ ]:
# ftpn_list = []

# metrics = eval("scenario.metrics_"+model+".read()")

In [ ]:
# feature_selection_info = scenario.feature_selection_info.id

In [ ]:
# feature_selection_info_id = scenario.feature_selection_info.id
# threshold_baseline_id = scenario.threshold_baseline.id
# train_parameters_info_id = scenario.train_parameters_info.id


In [ ]:
# DATANODE_metrics_attr = print_object_attributes(DATANODE_metrics)

In [ ]:
# all_scenarios[0].id

In [ ]:
# all_jobs = tp.get_jobs()

# # print_object_attributes(all_jobs[0])
# for job in all_jobs : 
#     print(f"Job ID: {job.id}, Status: {job.status}")

In [ ]:
# all_jobs_status = [(job.id, job.status) for job in all_jobs]

In [ ]:
# all_jobs_status

In [ ]:
# scenarios = tp.get_entities_by_config_id(tp.get_scenarios()[0])
# print(scenarios)
# scenario = scenarios[0]

In [ ]:
# print_object_attributes(scenario)

In [ ]:
# print(scenario.get_inputs())
# print(scenario.get_outputs())

In [ ]:
# input_nodes = scenario.get_inputs()
# output_nodes = scenario.get_outputs()

In [ ]:
# input_node_1 = list(input_nodes)[0]
# output_node_1 = list(output_nodes)[0]
# # output_node_1 = list(output_node_1)[0].read()


In [ ]:
# list(output_nodes)[0]
# print_object_attributes(list(output_nodes)[0])

In [ ]:
# for o_n in list(output_nodes):
#     # print(o_n._config_id )
#     if o_n._config_id.startswith('results_'): 
#         print(o_n._config_id)

# # o_n_1.read()

In [ ]:
# print_object_attributes(output_node_1)

In [ ]:
# models = {
#     "baseline": train_model_baseline,
#     "logistic_regression": train_model_logistic_regression,
#     "random_forest": train_model_rf_cv,
#     "xgboost": train_model_xgboost_cv,
#     # "tree": train_model_decision_tree,
# }

In [ ]:
# train_parameters_default_list = {}

# for model_name in models:
#     # print(models[model_name].__name__)
#     function_name = models[model_name].__name__
#     train_parameters = get_train_parameters_default(function_name)
#     train_parameters_default_list[function_name] = train_parameters


In [ ]:
# train_parameters_default_list.get("train_model_baseline")